# МУХА

> Бык с плугом на покой тащился по трудах \
А Муха у него сидела на рогах,\
И Муху же они дорогой повстречали.\
«Откуда ты, сестра?» — от этой был вопрос.\
А та, поднявши нос,\
В ответ ей говорит: «Откуда? — мы пахали!»\
От басни завсегда\
Нечаянно дойдешь до были.\
Случалось ли подчас вам слышать, господа:\
«Мы сбили! Мы решили!»\
*(И.И. Дмитриев)*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
df['totwrk2_10000'] = df['totwrk']**2/10000

In [3]:
model = smf.ols(data = df , formula = 'sleep~I(totwrk/100)+totwrk2_10000 +  age + smsa + male + south').fit()
model_1 = smf.ols(data = df , formula = 'I(totwrk/100)~totwrk2_10000 +  age + smsa + male + south').fit()
model_2 = smf.ols(data = df , formula = 'totwrk2_10000~ I(totwrk/100)+  age + smsa + male + south').fit()
model_3 = smf.ols(data = df , formula = 'male~I(totwrk/100)+ I(totwrk/100)+  age + smsa  + south').fit()
model_1.params.round(3)

Intercept        9.003
totwrk2_10000    0.021
age             -0.008
smsa             0.357
male             1.071
south            0.989
dtype: float64

In [4]:
model_2.params.round(3)

Intercept         -296.595
I(totwrk / 100)     40.125
age                  0.038
smsa               -19.824
male                -2.883
south              -37.165
dtype: float64

In [5]:
model_3.params.round(3)

Intercept          0.067
I(totwrk / 100)    0.020
age                0.002
smsa               0.008
south             -0.065
dtype: float64

In [6]:
model.params

Intercept          3414.144351
I(totwrk / 100)      -9.403265
totwrk2_10000        -0.189875
age                   2.838689
smsa                -60.523328
male                 90.709723
south                92.451937
dtype: float64

In [7]:
model.rsquared

0.1341373804431093

In [8]:
VIF_1 = 1/(1 - model_1.rsquared.round(3))
VIF_1.round(3)

8.475

In [9]:
VIF_2 = 1/(1 - model_2.rsquared.round(3))
VIF_2.round(3)

8.264

In [10]:
VIF_3 = 1/(1 - model_3.rsquared.round(3))
VIF_3.round(3)

1.172

## Рассмотрим первую модель в деталях

In [11]:
X = df[['totwrk2_10000' , 'age',  'smsa', 'male', 'south']]
X

,totwrk2_10000,age,smsa,male,south
0,1181.9844,32,0,1,0
1,2520.0400,31,0,1,1
2,792.4225,44,1,1,0
3,1433.3796,30,0,0,0
4,665.6400,64,0,1,0
...,...,...,...,...,...
701,410.4676,45,1,0,0
702,45.5625,34,0,1,0
703,342.6201,37,0,0,0
704,384.5521,54,0,0,1


## Формула для вычисления коэффициентов регрессии известна
## $$\hat{\beta} = (X' X)^{-1}(X')Y$$

In [12]:
np.linalg.inv(X.T@X).round(2)

array([[ 0.  , -0.  , -0.  , -0.  , -0.  ],
       [-0.  ,  0.  , -0.  , -0.  , -0.  ],
       [-0.  , -0.  ,  0.01, -0.  ,  0.  ],
       [-0.  , -0.  , -0.  ,  0.01,  0.  ],
       [-0.  , -0.  ,  0.  ,  0.  ,  0.01]])

In [13]:
np.linalg.det(np.linalg.inv(X.T@X).round(5))

0.0

## <span style="color:red"> Вывод.</span>
## Из курса линейной алгебры известно, что определитель матрицы равен нулю, если её столбцы/строки являются линейно зависимыми. Равенство нулю определителя матрицы $(X' X)^{-1}$ указывает на то, что в матрице есть зависимые параметры. В конкретно нашем примере это параметры $totwrk^2/1000$ и $totwrk/100$.